In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-164297
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-164297


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
#choose a name for my compute cluster
cpu_cluster_name = "my-cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("An existing cluster has been found, please use it.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
print(cpu_cluster.get_status().serialize())

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-11-23T20:00:52.356000+00:00', 'errors': None, 'creationTime': '2021-11-23T20:00:51.951023+00:00', 'modifiedTime': '2021-11-23T20:01:07.544440+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1,1),
    "--max_iter": choice(10,20,50,100,150,200,250)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1,delay_evaluation=4)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", compute_target=cpu_cluster, vm_size="STANDARD_D2_V2", entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

my_hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(my_hyperdrive_run).show()
my_hyperdrive_run.get_status()
my_hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021
Web View: https://ml.azure.com/runs/HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-164287/workspaces/quick-starts-ws-164287&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-23T14:38:29.168058][API][INFO]Experiment created<END>\n""<START>[2021-11-23T14:38:30.048626][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-11-23T14:38:30.262645][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021
Web View: https://ml.azure.com/runs/HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-164287/workspaces/quick-starts-ws-164287&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021',
 'target': 'my-cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-11-23T14:38:28.902599Z',
 'endTimeUtc': '2021-11-23T14:58:23.955955Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5d6fbfb2-3697-4ab7-9e8e-de6affca55ca',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'score': '0.9089529590288316',
  'best_child_run_id': 'HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021_14',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg164287.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021/azureml-logs/hyperdrive.txt

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
my_hyperdrive_best_run = my_hyperdrive_run.get_best_run_by_primary_metric()

print("Details of my best run: ", my_hyperdrive_best_run.get_details(),"/n")
print("File names of my best run: ", my_hyperdrive_best_run.get_file_names(), "/n")
print("Metrics of my best run: ", my_hyperdrive_best_run.get_metrics())

my_model = my_hyperdrive_best_run.register_model(model_name="register_my_hyperdrive_best_run_log",
model_path="./", tags={"Method" : "HyperDrive"},
properties={"Accuracy": my_hyperdrive_best_run.get_metrics()["Accuracy"]})
joblib.dump(value=my_hyperdrive_best_run.id, filename="./outputs/my_best_hyperdrive_model.joblib")


Details of my best run:  {'runId': 'HD_c91c1a92-7b2c-4ef0-86a1-d5ec53e36021_14', 'target': 'my-cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2021-11-23T14:50:36.411226Z', 'endTimeUtc': '2021-11-23T14:52:07.251037Z', 'services': {}, 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '5d6fbfb2-3697-4ab7-9e8e-de6affca55ca', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath

['./outputs/my_best_hyperdrive_model.joblib']

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(web_link)

In [7]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset, Datastore, Workspace

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

concat_data = pd.concat([x,y], axis=1)

my_train_data, my_test_data = train_test_split(concat_data, test_size=0.2, random_state=66)

ws = Workspace.from_config()

my_datastore = ws.get_default_datastore()

final_training_data = Dataset.Tabular.register_pandas_dataframe(my_train_data, my_datastore, "data_from_pd_to_tab", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/26747e2b-ed8f-46bb-b124-20448cf00956/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_training_data,
    label_column_name='y',
    n_cross_validations=6,
    compute_target=cpu_cluster,
    enable_early_stopping=True,
    enable_onnx_compatible_models=True)

In [10]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, name="my_automl_regression")
my_remote_run = exp.submit(config=automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on my-cpu-cluster with default configuration
Running on remote compute: my-cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6dcd4786-f269-4657-b233-32372aedaae3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [11]:
my_automl_best_run, my_automl_fitted_model = my_remote_run.get_output()
print(hasattr(my_automl_fitted_model, 'steps'))

True


In [15]:
my_automl_best_run.get_metrics()

{'average_precision_score_weighted': 0.9563752740550294,
 'recall_score_micro': 0.9176782189206474,
 'recall_score_macro': 0.7560258068464494,
 'average_precision_score_macro': 0.8280112915585133,
 'f1_score_weighted': 0.9143282210909142,
 'AUC_weighted': 0.9490406750660109,
 'precision_score_micro': 0.9176782189206474,
 'f1_score_micro': 0.9176782189206474,
 'matthews_correlation': 0.5553592324789198,
 'f1_score_macro': 0.7759007400233422,
 'average_precision_score_micro': 0.9818770654203615,
 'accuracy': 0.9176782189206474,
 'weighted_accuracy': 0.9577060633682676,
 'precision_score_weighted': 0.9123973618774078,
 'AUC_micro': 0.9812564234307439,
 'AUC_macro': 0.9490406750660111,
 'recall_score_weighted': 0.9176782189206474,
 'log_loss': 0.20227354813711532,
 'precision_score_macro': 0.8013106744438702,
 'balanced_accuracy': 0.7560258068464494,
 'norm_macro_recall': 0.5120516136928989,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_6dcd4786-f269-4657-b233-32372aedaae3

In [16]:
print(my_automl_best_run)

Run(Experiment: udacity-project,
Id: AutoML_6dcd4786-f269-4657-b233-32372aedaae3_30,
Type: azureml.scriptrun,
Status: Completed)


In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from pprint import pprint

# Print model to print the hyperparameters from each automl training pipeline step
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' _ ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' _ ')
        else:
            pprint(step[1].get_params())
            print()

# Get the best model
print_model(my_automl_fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': False,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': True,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/project-compute/code/Users/odl_user_164287'}

prefittedsoftvotingclassifier
{'estimators': ['0', '14', '1', '18', '23', '4', '12', '22', '20'],
 'weights': [0.23076923076923078,
             0.15384615384615385,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693]}

0 _ maxabsscaler
{'copy': True}

0 _ lightgbmclassifier
{'min_data_in_leaf': 20,
 'n_jobs': 1,
 'problem_info': ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'c

In [18]:
my_automl_best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_6dcd4786-f269-4657-b233-32372aedaae3_ModelExplain',
 'model_explanation': 'True'}

In [19]:
# Save my best automl run
import joblib
joblib.dump(value=my_automl_best_run.id, filename="./my_best_automl_model.joblib")

['./my_best_automl_model.joblib']

In [8]:
#delete compute cluster